In [1]:
import cv2
import numpy as np

In [3]:
# Tải mô hình đã được huấn luyện từ trước
net = cv2.dnn.readNetFromCaffe(
    './models/deploy.prototxt.txt',
    './models/res10_300x300_ssd_iter_140000_fp16.caffemodel'
)

# Mở webcam
cap = cv2.VideoCapture(0)

while True: 
    # Đọc khung hình từ camera
    ret, frame = cap.read()
    
    if not ret:
        break
    
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104, 177, 123), swapRB=False)

    # Đặt dữ liệu đầu vào cho mạng
    net.setInput(blob)

    # Chạy mạng để phát hiện khuôn mặt
    faces = net.forward()

    # Lấy kích thước của ảnh đầu vào
    h = frame.shape[0]
    w = frame.shape[1]

    # Duyệt từng khuôn mặt đã được phát hiện
    for i in range(0, faces.shape[2]):
        confidence = faces[0,0,i,2]
        # Kiểm tra nếu mặt có độ tin cậy là 0.5
        if confidence>0.5:
            # Trích xuất tọa độ
            # print(faces[0, 0, i, 3:7])
            startx = int(faces[0, 0, i, 3]*w)
            starty = int(faces[0, 0, i, 4]*h)
            endx = int(faces[0, 0, i, 5]*w)
            endy = int(faces[0, 0, i, 6]*h)
            # print(startx, starty, endx, endy)

            # Vẽ hình chữ nhật xung quanh khuôn mặt đã phát hiện
            cv2.rectangle(frame, (startx, starty), (endx, endy), (0, 255, 0),)

            # Hiển thị độ tin cậy
            text = 'Face: {:.2f}%'.format(confidence*100);
            cv2.putText(frame, text, (startx, starty-10), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255),)

    # Hiển thị ảnh gốc
    cv2.imshow('Kết quả', frame)
    if(cv2.waitKey(1)==ord('q')):
        break

cap.release()
cv2.destroyAllWindows()